In [54]:
import urllib.request
import json
import pandas as pd
import requests
from PIL import Image
import shutil
import os
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import KMeans



url = (
    "https://pixabay.com/api/?key=42732485-b4c1847a3529ea910e8e5fdeb&q=animaux&image_type=photo&per_page=100"
)
response = urllib.request.urlopen(url)
responsedata = json.loads(response.read().decode("utf-8"))
responsedata

def download_image(url):

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        image_link = os.path.basename(url)
        filename = os.path.join("./images/", image_link)
        with open(filename, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

array = []

for result in responsedata["hits"]:
    array.append(
        (
            result["id"],
            result["tags"],
            result["largeImageURL"],
            result["imageWidth"],
            result["imageHeight"],
            result["imageSize"],
            result["likes"], 
            result["user"],
            result["largeImageURL"].split('.')[-1] #format
        )
    )

dataframe = pd.DataFrame(array, columns=["id", "tags", "largeImageURL", "imageWidth","imageHeight","imageSize", "likes", "user", "format"])
dataframe = dataframe.astype(
    dtype={"id": "<U200", "tags": "<U200", "largeImageURL": "<U200","imageWidth": "<U200","imageHeight": "<U200","imageSize": "<U200", "likes": "<U200", "user": "<U200", "format": "<U200"}
)
#dataframe.largeImageURL.apply(download_image)
dataframe 

,id,tags,largeImageURL,imageWidth,imageHeight,imageSize,likes,user,format
0,715588,"fox, nature, animals",https://pixabay.com/get/g0f2147417648984810322...,3595,2414,2226561,1507,AlainAudet,jpg
1,430441,"horses, galloping, running",https://pixabay.com/get/g366bdc0f9c7ada058351c...,4500,3000,1347483,1520,SorcerySoapHP,jpg
2,591359,"tigers, cub, snow",https://pixabay.com/get/ge01ed8521d87dd87e7f5e...,4342,2798,5424078,1746,Sponchia,jpg
3,2904536,"horses, pair, wild horses",https://pixabay.com/get/g8f92f79d8869daad37201...,4000,3000,2640822,1588,susannp4,jpg
4,2934720,"cat, kitten, pets",https://pixabay.com/get/ga7d87de6a2fad4e94993a...,3495,2330,1971228,1548,Kirgiz03,jpg
...,...,...,...,...,...,...,...,...,...
95,2222908,"giraffe, head, ossicones",https://pixabay.com/get/g684e16a20d8e003528e77...,4000,2669,3727985,689,Alexas_Fotos,jpg
96,2045655,"eagle, portrait, wild",https://pixabay.com/get/g7e6bbcf6d5ba3005f7ec3...,2066,2426,1388406,851,Robert_C,jpg
97,2107052,"swan, water, white",https://pixabay.com/get/g6092bd4b762c6af997919...,4000,2289,1321269,1277,NoName_13,jpg
98,1341881,"wolves, forest, wintry",https://pixabay.com/get/g6280b3db60e0d240fe87d...,4096,2160,1380594,3163,51581,jpg


On va stocker id, tags, largeImageURL, imageWidth, imageHeight, imageSize, likes, format de l'image en prenant ce qu'il y a après le .; user

Le code suivant va obtenir les couleurs prédominantes de chaque image.

In [65]:
all_images=os.listdir("./images") #list of all the images' names
image_color=[] #list to add the images' color into the dataframe
dataframe['color']='None'
colorsOfAllImages=[] 
for i in range(len(all_images)):
    colorsOfOneImage="" #Reinitialization of the value of the colors
    color = [] #most significant colors of the images
    imgfile = Image.open("./images/"+all_images[i])
    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    if numarray.ndim == 1:
        imgfile = imgfile.convert('RGB')
        numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    clusters = KMeans(n_clusters=4, n_init=2)
    clusters.fit(numarray)
    npbins = numpy.arange(0, 5)
    histogram = numpy.histogram(clusters.labels_, bins=npbins)
    labels = numpy.unique(clusters.labels_)
    for i in range(4):
        image_color=[math.ceil(clusters.cluster_centers_[i][0]),math.ceil(clusters.cluster_centers_[i][1]),math.ceil(clusters.cluster_centers_[i][2])]
        color.append(''.join(format(i, '02x') for i in image_color))
    colorsOfOneImage=', '.join(color)
    colorsOfAllImages.append(colorsOfOneImage)
dataframe.loc[:,'color']=colorsOfAllImages
dataframe




,id,tags,largeImageURL,imageWidth,imageHeight,imageSize,likes,user,format,color
0,715588,"fox, nature, animals",https://pixabay.com/get/g0f2147417648984810322...,3595,2414,2226561,1507,AlainAudet,jpg,"cab13e, 3ca3bb, 393930, efc585"
1,430441,"horses, galloping, running",https://pixabay.com/get/g366bdc0f9c7ada058351c...,4500,3000,1347483,1520,SorcerySoapHP,jpg,"765a4e, eceeef, a39a80, 362823"
2,591359,"tigers, cub, snow",https://pixabay.com/get/ge01ed8521d87dd87e7f5e...,4342,2798,5424078,1746,Sponchia,jpg,"a7150c, b6b57f, 697b39, eae2cb"
3,2904536,"horses, pair, wild horses",https://pixabay.com/get/g8f92f79d8869daad37201...,4000,3000,2640822,1588,susannp4,jpg,"010101, 993a03, d7650b, fa9630"
4,2934720,"cat, kitten, pets",https://pixabay.com/get/ga7d87de6a2fad4e94993a...,3495,2330,1971228,1548,Kirgiz03,jpg,"826727, 2e240c, be9b76, 584617"
...,...,...,...,...,...,...,...,...,...,...
95,2222908,"giraffe, head, ossicones",https://pixabay.com/get/g684e16a20d8e003528e77...,4000,2669,3727985,689,Alexas_Fotos,jpg,"3c312b, 656064, aa7432, d5a682"
96,2045655,"eagle, portrait, wild",https://pixabay.com/get/g7e6bbcf6d5ba3005f7ec3...,2066,2426,1388406,851,Robert_C,jpg,"2f261a, d1c096, 8b7459, f1dfb6"
97,2107052,"swan, water, white",https://pixabay.com/get/g6092bd4b762c6af997919...,4000,2289,1321269,1277,NoName_13,jpg,"b1d09f, 20440b, 6f867c, 56ac0f"
98,1341881,"wolves, forest, wintry",https://pixabay.com/get/g6280b3db60e0d240fe87d...,4096,2160,1380594,3163,51581,jpg,"4d4b4b, e3e3e2, 020202, 9f9e9e"
